In [2]:
import abc

In [12]:
def overload(f):
    f.__overload__ = True
    return f

_MISSING = object()

class OverloadList(list):
    pass

class OverloadDict(dict):
    def __setitem__(self, __key, __value) -> None:


        prior_value = self.get(__key, _MISSING)
        overloaded = getattr(__value, '__overload__', False) 

        if prior_value is _MISSING:
            insert_val = OverloadList([__value]) if overloaded else __value
            super().__setitem__(__key, insert_val)
        
        elif isinstance(prior_value, OverloadList):
            if not overloaded:
                raise AttributeError 
            prior_value.append(__value)
        else:
            if overloaded:
                raise AttributeError
            
            super().__setitem__(__key, __value)

In [27]:
@overload
def foo():
    print('test')



o = OverloadDict()
o['a'] = 1
o['f'] = foo
o['f'] = foo



__main__.OverloadList

In [23]:
from typing import Any, Mapping

class OverloadMeta(type):
    @classmethod
    def __prepare__(cls, __name: str, __bases: tuple, **kwds: Any) -> Mapping[str, object]:
        return OverloadDict()

    def __new__(cls, name, bases, namespace, **kwargs):
        overload_namespace = {
            key: Overload(val) if isinstance(val, OverloadList) else val for key, val in namespace.items()
        }

        return super().__new__(cls, name, bases, overload_namespace, **kwargs)

In [44]:
class DummyMeta(type):
    def __new__(cls, name, bases, namespace, **kwargs):
        print(f'cls: {cls}')
        print(f'name: {name}')
        print(f'bases: {bases}')
        print(f'namespace: {namespace}')

        return super().__new__(cls, name, bases, namespace, **kwargs)



class TestClass(metaclass=DummyMeta):
    
    def __init__(self) -> None:
        pass

    def __set_name__(self, owner, name):
        print(f'__set_name__: {owner}, {name}')
        self.name = name
        self.owner = owner

    def foo(self, a:str):
        print('foo')

    def baz(self, b:int):
        print(b)

cls: <class '__main__.DummyMeta'>
name: TestClass
bases: ()
namespace: {'__module__': '__main__', '__qualname__': 'TestClass', '__init__': <function TestClass.__init__ at 0x7fdf4135f310>, '__set_name__': <function TestClass.__set_name__ at 0x7fdf41a105e0>, 'foo': <function TestClass.foo at 0x7fdf42690f70>, 'baz': <function TestClass.baz at 0x7fdf42690ee0>}


In [46]:
t = TestClass()

<bound method TestClass.__set_name__ of <__main__.TestClass object at 0x7fdf41459f70>>

In [22]:
import inspect

def foo(a:int, b:str) -> list:
    return []

inspect.signature(foo)

<Signature (a: int, b: str) -> list>

In [36]:

o = OverloadDict()

@overload
def foo(a, b):
    return 'test'

o['f'] = foo

@overload
def foo(a, b, c):
    return 'test2'

o['f'] = foo

o

{'f': [<function __main__.foo(a, b)>, <function __main__.foo(a, b, c)>]}

In [37]:
(overload_list := o['f'])

[<function __main__.foo(a, b)>, <function __main__.foo(a, b, c)>]

In [47]:
(signatures := [inspect.signature(f) for f in overload_list]) # THIS IS HOW WE MATCH

[<Signature (a, b)>, <Signature (a, b, c)>]

In [49]:
help(inspect.signature(overload_list[0]).bind)

Help on method bind in module inspect:

bind(*args, **kwargs) method of inspect.Signature instance
    Get a BoundArguments object, that maps the passed `args`
    and `kwargs` to the function's signature.  Raises `TypeError`
    if the passed arguments can not be bound.



In [51]:
def _type_hint_matches(obj, hint):
    # only works with concrete types, not things like Optional
    return hint is inspect.Parameter.empty or isinstance(obj, hint)


def _signature_matches(sig: inspect.Signature,
                       bound_args: inspect.BoundArguments):
    # doesn't handle type hints on *args or **kwargs
    for name, arg in bound_args.arguments.items():
        param = sig.parameters[name]
        hint = param.annotation
        if not _type_hint_matches(arg, hint):
            return False
    return True


class NoMatchingOverload(Exception):
    pass


class BoundOverloadDispatcher:
    def __init__(self, instance, owner_cls, name, overload_list, signatures):
        self.instance = instance
        self.owner_cls = owner_cls
        self.name = name
        self.overload_list = overload_list
        self.signatures = signatures

    def best_match(self, *args, **kwargs):
        for f, sig in zip(self.overload_list, self.signatures):
            try:
                bound_args = sig.bind(self.instance, *args, **kwargs)
            except TypeError:
                pass  # missing/extra/unexpected args or kwargs
            else:
                bound_args.apply_defaults()
                # just for demonstration, use the first one that matches
                if _signature_matches(sig, bound_args):
                    return f

        raise NoMatchingOverload()

    def __call__(self, *args, **kwargs):
        try:
            f = self.best_match(*args, **kwargs)
        except NoMatchingOverload:
            pass
        else:
            return f(self.instance, *args, **kwargs)

        # no matching overload in owner class, check next in line
        super_instance = super(self.owner_cls, self.instance)
        super_call = getattr(super_instance, self.name, _MISSING)
        if super_call is not _MISSING:
            return super_call(*args, **kwargs)
        else:
            raise NoMatchingOverload()


In [62]:
class Overload:
    def __init__(self, overload_list) -> None:
        self.overload_list = overload_list
        self.signatures = [inspect.signature(f) for f in overload_list]

    def __set_name__(self, owner, name):
        self.owner = owner
        self.name = name

    def __get__(self, instance, _owner=None):
        print(f'Overload: __get__ instance: {instance}, owner: {self.owner}')
        if instance is None:
            return self

     
        return BoundOverloadDispatcher(instance, self.owner, self.name, self.overload_list, self.signatures)

In [71]:
class OverloadMeta(type):

    @classmethod
    def __prepare__(mcs, name, bases):
        return OverloadDict()

    def __new__(mcs, name, bases, namespace, **kwargs):
        print('namespace: ', namespace)
        overload_namespace = {
            key: Overload(val) if isinstance(val, OverloadList) else val
            for key, val in namespace.items()
        }
        return super().__new__(mcs, name, bases, overload_namespace, **kwargs)


In [72]:
class A(metaclass=OverloadMeta):
    @overload
    def f(self, x:int):
        print(f'A.f int overload', self, x)

    @overload
    def f(self, x:str):
        print(f'A.f str overload', self, x)

namespace:  {'__module__': '__main__', '__qualname__': 'A', 'f': [<function A.f at 0x7fdf41b9cca0>, <function A.f at 0x7fdf41b9c5e0>]}


In [69]:
a = A()

In [70]:
a.f(1)
a.f('1')

Overload: __get__ instance: <__main__.A object at 0x7fdf40fa7160>, owner: <class '__main__.A'>
A.f int overload <__main__.A object at 0x7fdf40fa7160> 1
Overload: __get__ instance: <__main__.A object at 0x7fdf40fa7160>, owner: <class '__main__.A'>
A.f str overload <__main__.A object at 0x7fdf40fa7160> 1
